### 단일 리뷰 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome('../chromedriver.exe')

In [ ]:
corp = "SK하이닉스"
url = "https://www.jobplanet.co.kr/companies/20561/reviews/%EC%97%90%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4"

In [ ]:
# url 들어가기
driver.get(url)
driver.implicitly_wait(10) # 페이지가 전부 로딩될때까지 최대 10초 기다리기

In [ ]:
# 팝업 창이 뜨면 끄기 
try:
    driver.find_element(By.CLASS_NAME, "btn_close_x_ty1").click() 
except:
    pass

In [ ]:
review_list = driver.find_elements(By.CLASS_NAME, "content_wrap")

In [ ]:
review = review_list[0]

In [ ]:
date = review.find_elements(By.CLASS_NAME, "txt1")[-1].get_attribute('innerHTML')
print(date)

In [ ]:
score_list = review.find_element(By.CLASS_NAME, "ctbody_lft") 

In [ ]:
star_score = score_list.find_element(By.CLASS_NAME, "star_score").get_attribute('style')
print(star_score)

In [ ]:
# 항목별 평점
bl_score_list = score_list.find_elements(By.CLASS_NAME, "bl_score")
welfare_salary = bl_score_list[0].get_attribute("style")
work_life_balance = bl_score_list[1].get_attribute("style")
culture = bl_score_list[2].get_attribute("style")
promotion_opp = bl_score_list[3].get_attribute("style")
management = bl_score_list[4].get_attribute("style")
print(welfare_salary, work_life_balance, culture, promotion_opp, management)

In [ ]:
def convert_width_to_score(width):
    width_percent = int(width.split(' ')[1][:-2])
    score = int(width_percent / 20)
    return score

In [ ]:
print(convert_width_to_score(star_score))

In [ ]:
def extract_score_from_review(review):
    # 출력 초기화
    output = []
    # 리뷰에서 평점과 관련된 부분만 가져오기
    score_list = review.find_element(By.CLASS_NAME, "ctbody_lft")
    # 총 평점 가져와서 추가
    star_score = score_list.find_element(By.CLASS_NAME, "star_score").get_attribute('style')
    output.append(convert_width_to_score(star_score))
    # 항목별 평점 가져와서 추가
    other_score_elements = score_list.find_elements(By.CLASS_NAME, "bl_score")
    for element in other_score_elements:
        output.append(convert_width_to_score(element.get_attribute("style")))
    return output

In [ ]:
print(extract_score_from_review(review))

### 전체 리뷰 크롤링

#### 전체 페이지 수 획득

In [ ]:
btn_pglast = driver.find_element(By.CLASS_NAME, "btn_pglast")
num_pages = int(btn_pglast.get_attribute("href").split('=')[1])

#### 다음 페이지로 이동

In [ ]:
import time
def get_url(driver, url):
    while True:
        try:
            driver.get(url)
            driver.implicitly_wait(10) # 페이지가 전부 로딩될때까지 최대 10초 기다리기
            break
        except:
            driver.close()
            driver = webdriver.Chrome('../chromedriver.exe')
            time.sleep(60)

#### 리뷰 수집

In [ ]:
import pandas as pd
input_data = pd.read_excel("../../데이터/종목별_잡플래닛url.xlsx")
data = [] 

for corp, url in input_data[['종목명', 'URL']].values:
    # 초기 url 들어가기
    get_url(driver, url)
    # 전체 페이지 수 계산
    btn_pglast = driver.find_element(By.CLASS_NAME, "btn_pglast")
    num_pages = int(btn_pglast.get_attribute("href").split('=')[1])
    # 페이지 순회 시작
    for page in range(num_pages - 1):
        print("{}:{}/{}".format(corp, page+1, num_pages))
        current_url = url + "?page={}".format(page+1)
        get_url(driver, current_url)

        # 팝업 창이 뜨면 끄기 
        try:
            driver.find_element(By.CLASS_NAME, "btn_close_x_ty1").click()
            time.sleep(0.5)
        except:
            pass
        
        # 현재 페이지에서 데이터 가져오기
        while True:
            time.sleep(0.5)
            try:
                temp = []
                review_list = driver.find_elements(By.CLASS_NAME, "content_wrap")
                for review in review_list:
                    date = review.find_elements(By.CLASS_NAME, "txt1")[-1].get_attribute('innerHTML')
                    score = extract_score_from_review(review)
                    record = [corp, date] + score # score는 리스트임
                    temp.append(record)
                data += temp
                break
            except:
                driver.close()
                driver = webdriver.Chrome('../chromedriver.exe')
                get_url(driver, current_url)
                time.sleep(60)

data = pd.DataFrame(data, columns = ["기업", "날짜", "전체평점", "승진기회및가능성", "복지및급여", "업무와삶의균형", "사내문화", "경영진"])

In [ ]:
display(data.head())

In [ ]:
# 데이터 정제
data = data.groupby(['기업', '날짜'], as_index = False)[["전체평점", "승진기회및가능성", "복지및급여", "업무와삶의균형", "사내문화", "경영진"]].mean()
data.sort_values(by = ['기업', '날짜'], inplace = True)
display(data.head(10))

In [ ]:
data.to_csv("../../데이터/기업월별평점.csv", index = False, encoding = "euc-kr")